In [1]:
import os
from datetime import datetime
import pandas_datareader as pdr
from urllib import urlopen
from bs4 import BeautifulSoup
import json
import pandas as pd
import requests
import csv
from tqdm import tnrange, tqdm_notebook
from time import sleep
import pickle
import matplotlib.pyplot as plt
%pylab inline


no_hist_symbol = []
DIR = 'C:/Users/sarab/Desktop/git_projects/stock_analyst_grade_history/'
URL = 'https://finance.yahoo.com/quote/{0}/analysts?p={0}'

Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda2\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
##############################################################################
#### Populating Ticker Lists #################################################
##############################################################################

def locate_all_symbols(suffix,base_filename,dir_name):
    script_dir = os.path.join(dir_name,'.'.join((base_filename,suffix))) 
    all_companies = pd.read_csv(script_dir)
    all_ticker_symbols = all_companies['Symbol'].tolist()
    return all_ticker_symbols

def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)      
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)    
    return tickers    


entire_ticker_list = locate_all_symbols('csv','company_ticker_symbols', DIR)
snp500tickers = save_sp500_tickers()

In [3]:
def select_ticker(tick):
    url = 'https://finance.yahoo.com/quote/{0}/analysts?p={0}'
    return url.format(tick)

def yahoo_finance_analyst_data(url):
    req = urlopen(url)
    raw = req.read()
    soup = BeautifulSoup(raw,'lxml')
    return soup

def auto_symbols(ticker):
    url = select_ticker(ticker)
    return url

def grab_dict(soup):
    l = [i for i in soup.find_all('script')]
    mylist = [str(item) for item in l]
    m = max(mylist,key=len)
    ind = mylist.index(m)
    data = l[mylist.index(m)]
    data_dict = data.text[112:-12]
    all_tables = json.loads(data_dict)
    return all_tables

def upgrade_downgrade_pandas(dic,tick):
    try:
        stock_upgrade_downgrade_history = dic['context']['dispatcher']['stores']['QuoteSummaryStore']['upgradeDowngradeHistory']['history']
        pd_stock_upgrade_downgrade_history = pd.DataFrame(stock_upgrade_downgrade_history)
        pd_stock_upgrade_downgrade_history['epochGradeDate'] = pd.to_datetime(pd_stock_upgrade_downgrade_history['epochGradeDate'],unit='s')
        pd_stock_upgrade_downgrade_history.columns.values[1] = 'date'
        pd_stock_upgrade_downgrade_history.insert(0,'stockTicker',tick)
        return pd_stock_upgrade_downgrade_history
    except Exception:
        print "error" + tick
        no_hist_symbol.append(tick)
        

def single_company_hist(tick):
    url = select_ticker(tick)
    soup = yahoo_finance_analyst_data(url)
    full_dict = grab_dict(soup)
    return upgrade_downgrade_pandas(full_dict,tick)

def list_of_all_stock_grading_history(good_list):
    
    all_stock_grading_history = []
    for ticker in good_list:
        if ticker not in no_hist_symbol:
            url = auto_symbols(ticker)
            soup = yahoo_finance_analyst_data(url)
            all_tables = grab_dict(soup)
            grading_history = upgrade_downgrade_pandas(all_tables,ticker)
            all_stock_grading_history.append(grading_history)
    concat_lists_to_pd = pd.concat(all_stock_grading_history)
    return concat_lists_to_pd


##############################################################################
##### Trading Data   #########################################################
##############################################################################

def get_stock_history(df):
    list_of_stock_histories = []
    for ticker in df['stockTicker'].unique():
        try:
            stock_df = pdr.get_data_yahoo(symbols=ticker, start=df.loc[df['stockTicker']==ticker].date.iloc[-1])
            stock_df['stockTicker'] = ticker
            stock_df.reset_index(inplace = True)
            stock_df['Percent_change'] =  stock_df.Close.pct_change()
            list_of_stock_histories.append(stock_df)
            print ticker
        except Exception: 
            print 'error @ ' + ticker
    concat_lists_to_pd = pd.concat(list_of_stock_histories)
    return concat_lists_to_pd
    

## S&P 500 Dataset

In [ ]:
# snp500data = list_of_all_stock_grading_history(snp500tickers)
# snp500data.to_csv('snp500analyst_history')

## All Stock Ticker Dataset

In [ ]:
#all_ticker_data = list_of_all_stock_grading_history(entire_ticker_list)
#all_ticker_data.to_csv('entire_ticker_analyst_history')
#all_ticker_data.head()

In [4]:
all_ticker_data = pd.read_csv('C:\Users\sarab\Desktop\git_projects\stock_analyst_grade_history\entire_ticker_analyst_history')
snp_500_data_df = pd.read_csv('C:\Users\sarab\Desktop\git_projects\stock_analyst_grade_history\snp500analyst_history')

In [7]:
print len(all_ticker_data)
print len(snp_500_data_df)

49483
46513


In [12]:
snp_tickers = snp_500_data_df['stockTicker']

In [41]:
snp_500_data_df['stockTicker'].unique()

array(['MMM', 'ABT', 'ABBV', 'ACN', 'ATVI', 'AYI', 'ADBE', 'AMD', 'AAP',
       'AES', 'AET', 'AMG', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE',
       'ALXN', 'ALGN', 'ALLE', 'AGN', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG',
       'MO', 'AMZN', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK',
       'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'APC', 'ADI', 'ANDV', 'ANSS',
       'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM',
       'ARNC', 'AJG', 'AIZ', 'T', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY',
       'BHGE', 'BLL', 'BAC', 'BK', 'BAX', 'BBT', 'BDX', 'BBY', 'BIIB',
       'BLK', 'HRB', 'BA', 'BWA', 'BXP', 'BSX', 'BHF', 'BMY', 'AVGO',
       'CHRW', 'CA', 'COG', 'CDNS', 'CPB', 'COF', 'CAH', 'CBOE', 'KMX',
       'CCL', 'CAT', 'CBG', 'CBS', 'CELG', 'CNC', 'CNP', 'CTL', 'CERN',
       'CF', 'SCHW', 'CHTR', 'CHK', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'XEC',
       'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS',
       'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO'

In [10]:
len(snp_tickers)

503

In [53]:
snp_500_data_df.head()

,Unnamed: 0,stockTicker,action,date,firm,fromGrade,toGrade
0,0,MMM,main,2017-10-25,Citigroup,NaN,Buy
1,1,MMM,main,2017-10-25,Credit Suisse,NaN,Outperform
2,2,MMM,main,2017-10-25,Bank of America,NaN,Buy
3,3,MMM,main,2017-10-25,Argus,NaN,Buy
4,4,MMM,up,2017-10-25,Goldman Sachs,Sell,Neutral


### Single Company Lookup by Ticker

In [ ]:
df = single_company_hist(raw_input('Please Enter Ticker Value').upper())

In [54]:
snp500StockValueHistories.head()

,Date,Open,High,Low,Close,Adj Close,Volume,stockTicker,Percent_change
0,2012-02-09,87.540001,88.180000,87.459999,88.019997,75.877464,3031600.0,MMM,NaN
1,2012-02-10,87.199997,87.559998,86.750000,87.139999,75.118858,3085700.0,MMM,-0.009998
2,2012-02-13,87.570000,88.089996,87.160004,88.029999,75.886070,2530600.0,MMM,0.010213
3,2012-02-14,87.599998,88.000000,87.220001,87.989998,75.851593,2850300.0,MMM,-0.000454
4,2012-02-15,87.419998,87.720001,86.750000,87.010002,75.513138,3647000.0,MMM,-0.011138


## SNP500 Stock Value Histories

In [16]:
#snp500StockValueHistories = get_stock_history(snp_500_data_df)
#snp500StockValueHistories.to_csv('snp500stockHistories')

## Stock History and Analyst History Merged Data

In [61]:
entire_stock_analyst_hist_df = pd.read_csv('entire_ticker_analyst_history')
snp_500_analyst_history_df = pd.read_csv('snp500analyst_history')
snp_500_stock_history_df = pd.read_csv('snp500stockHistories')

In [62]:
snp500_stock_and_updown_hist_merged_df = snp_500_stock_history_df.merge(snp_500_analyst_history_df,how='left',left_on= ['Date','stockTicker'], right_on=['date','stockTicker'])

In [72]:
cols = list(snp500_stock_and_updown_hist_merged_df.columns.values)
cols

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'stockTicker',
 'Percent_change',
 'Unnamed: 0_y',
 'action',
 'firm',
 'fromGrade',
 'toGrade']

In [74]:
snp500_stock_and_updown_hist_merged_df = snp500_stock_and_updown_hist_merged_df.drop('Unnamed: 0_y',1)

In [76]:
cols = list(snp500_stock_and_updown_hist_merged_df.columns.values)
cols

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'stockTicker',
 'Percent_change',
 'action',
 'firm',
 'fromGrade',
 'toGrade']

In [77]:
snp500_stock_and_updown_hist_merged_df = snp500_stock_and_updown_hist_merged_df[['stockTicker','Date','firm','action','fromGrade','toGrade','Open', 'High','Low','Close','Adj Close','Volume']]

In [79]:
snp500_stock_and_updown_hist_merged_df.to_csv('snp500merged')

In [52]:
snp500_merged_df.firm.value_counts()

Series([], Name: firm, dtype: int64)